## Integration with Azure Data Lake Storage Gen2

In [0]:
tiers = ["bronze", "silver", "gold"] # Medallion tiers
# Azure Data Lake Storage Gen2 tier paths dictionary
adls_paths = {tier: f"abfss://{tier}@learnbucket.dfs.core.windows.net/"  for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

# Explore the pointers
dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

## Import libraries

In [0]:
import requests
import json
from datetime import date, timedelta

Define the time interval to fetch data from earthquake gov API

In [0]:
# set up start date: current day - 1 day interval
start_date = date.today() - timedelta(days=1)
# set up end date
end_date = date.today()

## Fetch data from Earthquake USGS API 

In [0]:
# URL for calling USGS API
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
  response = requests.get(url) # get data
  response.raise_for_status() # raise exception if request fails
  data = response.json().get('features', []) # extract data from json return
  # Handling no data case
  if not data:      
      print("No data returned for the specified date range")
  else:
      # Save data to bronze layer
      file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

      json_data = json.dumps(data, indent=4) # Convert data to JSON string format, indent=4 for human readability (pretty-printed)
      dbutils.fs.put(file_path, json_data) # Write a file to ADLS with the data
      print(f"Data saved to {file_path}")

except requests.exceptions.RequestException as e:
  print(f"Error fetching data from API: {e}") # Catch an exception if the request fails

## Next job information

In [0]:
# Create a structured output object
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Pass the output object to the next job task
dbutils.jobs.taskValues.set(key="bronze_output", value = output_data)